## IPMOF - InterPenetrating Metal Organic Frameworks

### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cutOff radius and grid size for energy map

### Read structural information for MOF files in a given directory
1. Reaf MOF files in ".mol2" format from given directory and create a list
2. Choose the first MOF1 as the stationary (map) MOF, initialize its variables, and calculate packed coordinates
3. For all the MOFs in the list:
4. Initialize MOF2 variables for the mobile MOF
5. Calculate energy map

In [27]:
import math
import os
os.chdir(r'/home/kutay/Documents/git/IPMOF/IPMOF_Python')
from forcefield import *
from crystal import *
from energymap import *
from visualize import *
from interpenetration import *
from quaternion import *
%pylab inline

# Read excel file containing force field information
excel_file_path = '/home/kutay/Documents/Research/FF_Parameters.xlsx'
uff = read_ff_parameters(excel_file_path, 'uff')

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### File Input Options
- <b> Built-in read_mol2 </b>
<pre>
uc_size, uc_angle, atom_names, atom_coors = read_mol2(mol2_path)
mof = MOF()
mof.initialize(mof.mol2_path)
</pre> 
- <b> Ase pdb read (cif read gives error) </b>
<pre>
from ase.io import read
mof_atoms = read(mof_dir, format='pdb')
mof_atoms.get_positions()                # Coordinates
mof_atoms.get_chemical_symbols()         # Atom names
mof_atoms.get_cell()                     # Cell vectors? (check)
mof_obj.get_number_of_atoms()            # Num of atoms
mof_obj.get_volume()                     # Unit cell volume

</pre>
- <b> Open babel </b>
<pre>
babel -icif *.cif -opdb *.pdb
</pre>

In [28]:
# Create list of MOFs
mol2_dir = r'/home/kutay/Documents/Research/MOFs/IPMOF_Python/mol2'
mol2_list = get_mof_list(mol2_dir, '.mol2')
print(mol2_list)

['QIGBIR.mol2', 'ZIF90.mol2', 'KINFAQ.mol2', 'NUVWIL.mol2']


In [3]:
# Read mol2 files and initialize MOF objects
base_mof = MOF()
base_mof.mol2_path = os.path.join(mol2_dir, mol2_list[0])
base_mof.initialize()
base_mof.initialize_ff(uff)

mobile_mof = MOF()
mobile_mof.mol2_path = os.path.join(mol2_dir, mol2_list[1])
mobile_mof.initialize()
mobile_mof.initialize_ff(uff)

In [4]:
# Calculate cut-off radius as Rc = L/2
width_a = base_mof.ucv / (base_mof.uc_size[1]*base_mof.uc_size[2] / math.sin(math.radians(base_mof.uc_angle[0])))
width_b = base_mof.ucv / (base_mof.uc_size[0]*base_mof.uc_size[2] / math.sin(math.radians(base_mof.uc_angle[1])))
width_c = base_mof.ucv / (base_mof.uc_size[0]*base_mof.uc_size[1] / math.sin(math.radians(base_mof.uc_angle[2])))
base_mof.cut_off = min(width_a / 2, width_b / 2, width_c / 2)

base_mof.packing_factor = Packing.factor(base_mof.uc_size, base_mof.cut_off)
uc_vectors = Packing.uc_vectors(base_mof.uc_size, base_mof.uc_angle)
trans_vec = Packing.translation_vectors(base_mof.packing_factor, uc_vectors)
base_mof.packed_coors = Packing.uc_coors(trans_vec, base_mof.packing_factor, uc_vectors, base_mof.atom_coors)
base_mof.edge_points = Packing.edge_points(uc_vectors)

In [5]:
atom_list = get_uniq_atom_list([mobile_mof])

In [6]:
emap = energy_map(base_mof, atom_list, base_mof.cut_off, 1)

In [7]:
init_coors = initial_coordinates(base_mof, emap, atom_list, 3E10)

0.0 0.0
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.0 16.5
0.

In [29]:
emap[-1]

array([  1.70000000e+01,   1.70000000e+01,   1.80000000e+01,
         9.26723432e+03,   7.24878895e+03,   7.51798936e+04,
         4.29632812e+04,   2.96175614e+04])

In [31]:
base_mof.uc_angle

[90.0, 90.0, 90.0]

In [32]:
17*17*18

5202